## Load Packages

In [1]:

include("printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

# Utility Function

Recall: the CRRA utility function is $U(W) = \frac{W^{1-k}}{1-k}$

In [3]:
U(x,k) =  x.^(1-k)/(1-k)    #CRRA utility function, k is the risk aversion

U (generic function with 1 method)

In [4]:
W = linspace(0.8,1.2,25)


plot(W,U(W,2),color=:red,linewidth=2,legend=false)
title!("Utility function, CRRA")
xlabel!("wealth")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 39.4926,369.674 62.683,348.473 85.8734,328.119 109.064,308.564 132.254,289.76 155.445,271.667 178.635,254.243 201.826,237.453 225.016,221.263 248.206,205.641 
 271.397,190.557 294.587,175.985 317.778,161.899 340.968,148.274 364.159,135.089 387.349,122.322 410.54,109.955 433.73,97.9676 456.92,86.3438 480.111,75.0669 
 503.301,64.1218 526.492,53.4939 549.682,43.1696 572.873,33.1362 596.063,23.3815 
 "/>

# Expected Utility

Recall: if $\pi_s$ is the probability of outcome $W_s$ and there are $S$ possible outcomes, then

$\text{E}W = \sum\nolimits_{s=1}^{S} \pi_{s}W_{s}$ and 

$\text{E}U(W) = \sum\nolimits_{s=1}^{S} \pi_{s}U(W_{s})$

Concavity of the utility function means that 
$\text{E} U(W) < U(\text{E} W)$

In [5]:
EU(x,π,k) = sum(π .* U(x,k))               #expected utility

EU (generic function with 1 method)

In [6]:
W₁ = 0.85               #possible outcomes
W₂ = 1.15
π₁ = 0.5                #probabilities of outcomes 
π₂ = 1 - π₁

xx = [W₁ U(W₁,2);
      W₂ U(W₂,2)]
println("Possible wealth levels and the corresponding utility")
printmat(xx)

UtilExpW = U(sum([π₁ π₂].*[W₁ W₂]),2)
ExpUtil  = EU([W₁ W₂],[π₁ π₂],2)
printlnPs("Utility at expected value and Expected utility: ",[UtilExpW ExpUtil])

Possible wealth levels and the corresponding utility
     0.850    -1.176
     1.150    -0.870

Utility at expected value and Expected utility:     -1.000    -1.023


In [7]:
plot(W,U(W,2),color=:red,linewidth=2,label="Utility fn")
scatter!([W₁;W₂],U([W₁;W₂],2),color=:green,label="possible outcomes")
scatter!([1.0],[ExpUtil],color=:blue,label="Expected utility")
plot!(ylims=(-1.3,-0.7))
title!("Utility function, CRRA")
xlabel!("wealth")
ylabel!("")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 1.1 
 
 
 1.2 
 
 
 -1.3 
 
 
 -1.2 
 
 
 -1.1 
 
 
 -1.0 
 
 
 -0.9 
 
 
 -0.8 
 
 
 -0.7 
 
 
 Utility function, CRRA 
 
 
 wealth 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 55.2446,340.816 77.1223,326.093 99.0001,311.959 120.878,298.379 142.756,285.321 164.633,272.756 186.511,260.656 208.389,248.996 230.267,237.753 252.144,226.904 
 274.022,216.43 295.9,206.31 317.778,196.528 339.656,187.066 361.533,177.91 383.411,169.044 405.289,160.456 427.167,152.131 449.044,144.059 470.922,136.228 
 492.8,128.627 514.678,121.247 536.555,114.077 558.433,107.109 580.311,100.335 
 "/>
 
 
 
 
 
 
 
 
 
 
 Utility fn 
 
 
 
 
 
 
 possible outcomes 
 
 
 
 
 
 
 Expected utility

# Certainty Equivalent

Recall: the certainty equivalent (here denoted $P$) is the sure (non-stochastic) value that solves 

$U(P) = \text{E}U(W)$

The code below solves for $P$ by a numerial search for different definitions of the $U$ function (different risk aversions).

Notice that the expected wealth level is 1.

In [8]:
using Roots

println("risk aversion and certainly equivalent")
kM = [0.0001;2;5;10;25]
for k in kM
    ceq = fzero(P->EU([W₁ W₂],[π₁ π₂],k) - P^(1-k)/(1-k),[0.5,1.5])
    printlnPs([k ceq])
end

risk aversion and certainly equivalent
     0.000     1.000
     2.000     0.977
     5.000     0.947
    10.000     0.912
    25.000     0.875


# Portfolio Choice with One Risky Asset

Recall: the investor solves $\max\nolimits_{v}\text{E}\ln R_{p}\text{, with }R_{p}=vR_{1} + (1-v)R_{f}$

In the example below, there are two possible outcomes for $R_{1}$ (-0.2 and 0.5) with equal probabilities

In [9]:
function EUc(v,π,Re,Rf)
    R = Re + Rf
    Rp = v*R + (1-v)*Rf             #portfolio return
    EU = sum(π .* log(1+Rp))        #expected utility
    return EU
end

EUc (generic function with 1 method)

In [10]:
    
v = linspace(-1,1,101)
L = length(v)
EUv = fill(NaN,L)
for i = 1:L
    EUv[i] = EUc(v[i],[0.5 0.5],[-0.3 0.4],0.1)
end    
    
plot(v,EUv,color=:red,linewidth=2,legend=false)
title!("Expected Utility")
xlabel!("weight on risky asset")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 Expected Utility 
 
 
 weight on risky asset 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 39.4926,369.674 45.0583,359.097 50.624,348.74 56.1897,338.596 61.7554,328.664 67.3211,318.939 72.8868,309.418 78.4525,300.096 84.0182,290.972 89.5839,282.041 
 95.1496,273.3 100.715,264.747 106.281,256.378 111.847,248.191 117.412,240.183 122.978,232.351 128.544,224.693 134.11,217.206 139.675,209.889 145.241,202.738 
 150.807,195.751 156.372,188.927 161.938,182.264 167.504,175.758 173.069,169.409 178.635,163.214 184.201,157.173 189.767,151.282 195.332,145.54 200.898,139.946 
 206.464,134.498 212.029,129.195 217.595,124.034 223.161,119.016 228.727,114.138 234.292,109.398 239.858,104.797 245.424,100.332 250.989,96.0024 256.555,91.807 
 262.121,87.7448 267.686,83.8146 273.252,80.0155 278.818,76.3465 284.384,72.8066 289.949,69.395 295.515,66.1109 301.081,62.9533 306.646,59.9215 312.212,57.0148 
 317.778,54.2324 323.343,51.5737 328.909,49.038 334.475,46.6247 340.041,44.3332 345.606,42.1629 351.172,40.1134 356.738,38.1842 362.303,36.3747 367.869,34.6845 
 373.435,33.1133 379.001,31.6607 384.566,30.3263 390.132,29.1099 395.698,28.011 401.263,27.0296 406.829,26.1652 412.395,25.4179 417.96,24.7873 423.526,24.2733 
 429.092,23.8758 434.658,23.5947 440.223,23.4299 445.789,23.3815 451.355,23.4493 456.92,23.6334 462.486,23.9339 468.052,24.3508 473.617,24.8843 479.183,25.5343 
 484.749,26.3012 490.315,27.185 495.88,28.186 501.446,29.3044 507.012,30.5405 512.577,31.8946 518.143,33.3669 523.709,34.9579 529.275,36.6679 534.84,38.4974 
 540.406,40.4466 545.972,42.5163 551.537,44.7067 557.103,47.0184 562.669,49.4521 568.234,52.0083 573.8,54.6876 579.366,57.4906 584.932,60.4181 590.497,63.4708 
 596.063,66.6494 
 "/>

Notice that this problem can be solved by pen and paper, but it 
becomes very difficult when the number of states increase --- and even worse when there are many assets. We therefore apply a numerical optimization algorithm.

In [11]:
using Optim          #minimization routines

#minimize -EU()
Sol = optimize(v-> -EUc(v,[0.5 0.5],[-0.3 0.4],0.1),-1,1)   #try v values between -1 and 1
printlnPs("argmin -EUc(): ",Optim.minimizer(Sol))

argmin -EUc():      0.458


# Portfolio Choice with Several Risky Assets

This optimization problem has two risky assets and a general CRRA utility function. Numerical optimization is still easy.

In [12]:
function EUe(v,π,R,Rf,γ)
    Rp = v'R + (1-sum(v))*Rf            #portfolio return
    EU = sum(π .* (1+Rp).^(1-γ)/(1-γ))  #expected utility
    EUNeg = -EU                         #since optimize() minimizes
    return EUNeg
end

EUe (generic function with 1 method)

In [13]:
ROutComes = [-0.03 0.08 0.20;           #cell[i,j] is asset i in state j
             -0.04 0.22 0.15]
Sol = optimize(v->EUe(v,[1/3 1/3 1/3],ROutComes,0.065,5),[-0.6;1.2])         #try v values between -1 and 1
printlnPs("optimal weights from max EUe(): ",Optim.minimizer(Sol))

optimal weights from max EUe(): 

# From Chapter on Mean-Variance Analysis

In [14]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF
    n   = length(μ)
    μe  = μ - Rf              #excess returns            
    Σ_1 = inv(Σ)
    w   = (mustar-Rf)/(μe'Σ_1*μe)[1] .* Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

# Mean-Variance and the Telser Criterion

Let $\mu$ be a vector of expected returns and $\Sigma$ be the covariance matrix of the investible asssets.

The Telser criterion solves the problem

$\max_{v} \mu_{p} \> \text{ subject to} \>  \text{VaR}_{95\%} < 0.1$,

where $\mu_{p} = v'\mu+(1-v)R_f$ is the expected portfolio return.

If the returns are normally distributed then 

$\text{VaR}_{95\%} = -(\mu_p - 1.64\sigma_p)$,

where $\sigma_p = \sqrt{v'\Sigma v}$ is the standard deviation of the portfolio return.

The figure below illustrates that the optimal portfolio is on the CLM (when the returns are normally distributed)

In [15]:
μ = [9; 6]/100                     #means
Σ = [ 256  0;
      0    144]/10000
Rf = 1/100

mustar  = linspace(Rf,0.1,101)
L       = length(mustar)
StdRp   = fill(NaN,L)
StdRpRf = fill(NaN,L)
for i = 1:L
    StdRp[i]   = MVCalc(mustar[i],μ,Σ)[1]
    StdRpRf[i] = MVCalcRf(mustar[i],μ,Σ,Rf)[1]
end   

VaRRestr = -0.1 + 1.64*StdRpRf


plot(StdRpRf*100,mustar*100,color=:blue,linewidth=2,label="CML",xlims=(0,15),ylims=(0,15))
plot!(StdRp*100,mustar*100,color=:red,linewidth=2,label="MVF")
plot!(StdRpRf*100,VaRRestr*100,line=(:dot,2),color=:green,label="VaR restriction")
title!("Mean vs standard deviation (with Rf)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation (with Rf) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 54.7703,346.588 59.7603,344.51 64.7503,342.432 69.7403,340.355 74.7303,338.277 79.7203,336.199 84.7103,334.121 89.7003,332.044 94.6903,329.966 99.6803,327.888 
 104.67,325.81 109.66,323.733 114.65,321.655 119.64,319.577 124.63,317.499 129.62,315.422 134.61,313.344 139.6,311.266 144.59,309.188 149.58,307.111 
 154.57,305.033 159.56,302.955 164.55,300.877 169.54,298.8 174.53,296.722 179.52,294.644 184.51,292.566 189.5,290.489 194.49,288.411 199.48,286.333 
 204.47,284.255 209.46,282.177 214.45,280.1 219.44,278.022 224.43,275.944 229.42,273.866 234.41,271.789 239.4,269.711 244.39,267.633 249.38,265.555 
 254.37,263.478 259.36,261.4 264.35,259.322 269.34,257.244 274.33,255.167 279.32,253.089 284.31,251.011 289.3,248.933 294.29,246.856 299.28,244.778 
 304.27,242.7 309.26,240.622 314.25,238.545 319.24,236.467 324.23,234.389 329.22,232.311 334.21,230.234 339.2,228.156 344.19,226.078 349.18,224 
 354.17,221.923 359.16,219.845 364.15,217.767 369.14,215.689 374.13,213.612 379.12,211.534 384.11,209.456 389.1,207.378 394.09,205.301 399.08,203.223 
 404.07,201.145 409.06,199.067 414.05,196.99 419.04,194.912 424.03,192.834 429.02,190.756 434.01,188.678 439,186.601 443.99,184.523 448.98,182.445 
 453.97,180.367 458.96,178.29 463.95,176.212 468.94,174.134 473.93,172.056 478.92,169.979 483.91,167.901 488.9,165.823 493.889,163.745 498.879,161.668 
 503.869,159.59 508.859,157.512 513.849,155.434 518.839,153.357 523.829,151.279 528.819,149.201 533.809,147.123 538.799,145.046 543.789,142.968 548.779,140.89 
 553.769,138.812 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 1557.93,346.588 1536.87,344.51 1515.82,342.432 1494.8,340.355 1473.79,338.277 1452.81,336.199 1431.84,334.121 1410.9,332.044 1389.97,329.966 1369.08,327.888 
 1348.2,325.81 1327.35,323.733 1306.53,321.655 1285.74,319.577 1264.98,317.499 1244.25,315.422 1223.56,313.344 1202.9,311.266 1182.27,309.188 1161.69,307.111 
 1141.14,305.033 1120.65,302.955 1100.19,300.877 1079.79,298.8 1059.44,296.722 1039.14,294.644 1018.91,292.566 998.733,290.489 978.627,288.411 958.591,286.333 
 938.632,284.255 918.754,282.177 898.964,280.1 879.267,278.022 859.671,275.944 840.183,273.866 820.81,271.789 801.563,269.711 782.452,267.633 763.486,265.555 
 744.678,263.478 726.042,261.4 707.592,259.322 689.345,257.244 671.317,255.167 653.53,253.089 636.006,251.011 618.768,248.933 601.843,246.856 585.263,244.778 
 569.06,242.7 553.271,240.622 537.936,238.545 523.1,236.467 508.813,234.389 495.127,232.311 482.1,230.234 469.794,228.156 458.276,226.078 447.615,224 
 437.881,221.923 429.148,219.845 421.487,217.767 414.966,215.689 409.648,213.612 405.589,211.534 402.831,209.456 401.406,207.378 401.331,205.301 402.606,203.223 
 405.217,201.145 409.134,199.067 414.315,196.99 420.705,194.912 428.242,192.834 436.86,190.756 446.486,188.678 457.048,186.601 468.475,184.523 480.696,182.445 
 493.646,180.367 507.262,178.29 521.485,176.212 536.262,174.134 551.544,172.056 567.284,169.979 583.443,167.901 599.984,165.823 616.871,163.745 634.076,161.668 
 651.57,159.59 669.329,157.512 687.33,155.434 705.554,153.357 723.983,151.279 742.599,149.201 761.388,147.123 780.337,145.046 799.433,142.968 818.665,140.89 
 838.024,138.812 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008100; stroke-width:1.6; stroke-opacity:1; fill:none" stroke-dasharray="1, 2" points="
 54.7703,600.536 59.7603,595.3 64.7503,590.065 69.7403,584.829 74.7303,579.594 79.7203,574.359 84.7103,569.123 89.7003,563.888 94.69